In [8]:
import adlframework
reload(adlframework)
from adlframework.retrievals.BlobLocalCache import BlobLocalCache
from adlframework.datasource import DataSource
from adlframework.dataentity.audio_de import AudioRecordingDataEntity
from adlframework.experiment import SimpleExperiment
from keras.losses import KLD, MAE
from keras.optimizers import Adadelta, Adam
# from nets import dense_network, deep_dense_network
import pdb
import numpy as np
from keras.callbacks import ModelCheckpoint
from functools import partial

In [9]:
cache_path = 'local_cache/AVEC/'
train_retrieval = BlobLocalCache(cache_path+'wav/train', cache_path+'labels/train')
val_retrieval = BlobLocalCache(cache_path+'wav/dev', cache_path+'labels/val')
test_retrieval = BlobLocalCache(cache_path+'wav/test', cache_path+'labels/test')
epochs = 100
max_mem = .95

Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.


In [10]:
input_window = 3 # seconds
universal_args = {
    'window_size': input_window,
    'timestamp_column': 'Timestamps',
    'sampling_method': 'linear_interpolation'
}

In [11]:
## Creating and splitting datasets
train_ds = DataSource(train_retrieval, AudioRecordingDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
                        max_mem_percent=max_mem,
                     **universal_args)

val_ds = DataSource(val_retrieval, AudioRecordingDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
                        max_mem_percent=max_mem,
                   **universal_args)

test_ds = DataSource(test_retrieval, AudioRecordingDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
                        max_mem_percent=max_mem,
                    **universal_args)

In [12]:
### Callbacks
callbacks = [ModelCheckpoint('weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5')]

In [13]:
a = train_ds.next()

# Net

## Train

In [14]:
batch = None
for _ in range(epochs):
    batch = train_ds.next()
    model.train_on_batch(*batch)

NameError: name 'model' is not defined